<a href="https://colab.research.google.com/github/tmckim/materials-sp24-colab/blob/main/lec_demos/lec11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**Copy of lec11.ipynb**". You can rename this to just the title of the assignment "**lec11.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them. <br>

I also recommend you give the folder where you save your notebooks^ a different name than the folder we create below that will store the notebook resources you need each time you work through a course notebook. This includes any data files you will need, links to the images that appear in the notebook, and the files associated with the autograder for answer checking.<br>
You should select a name other than '**NS499-DataSci-course-materials**'. <br>
This folder gets overwritten with each assignment you work on in the course, so you should **NOT** store your notebooks in this folder that we use for course materials! <br><br>For example, you could create a folder called 'NS499-**notebooks**' or something along those lines.
___

### We will now do the setup steps as separate cells to help with issues finding files in google drive/colab. <br> If you restart colab, you must rerun all **5** steps in each of these cells!

In [ ]:
# Step 1
# Setup and add files needed to access gdrive
from google.colab import drive                                   # these lines mount your gdrive to access the files we import below
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Step 2
# Change directory to the correct location in gdrive (modified way to do this from before)
import os
os.chdir('/content/gdrive/MyDrive/NS499-DataSci-course-materials/')

In [ ]:
# Step 3
# Remove the files that were previously there- we will replace with all the old + new ones for this assignment
!rm -r materials-sp24-colab

In [ ]:
# Step 4
# These lines clone (copy) all the files you will need from where I store the code+data for the course (github)
# Second part of the code copies the files to this location and folder in your own gdrive
!git clone https://github.com/tmckim/materials-sp24-colab '/content/gdrive/My Drive/NS499-DataSci-course-materials/materials-sp24-colab/'

In [ ]:
# Step 5
# Change directory into the folder where the resources for this assignment are stored in gdrive (modified way from before)
os.chdir('/content/gdrive/MyDrive/NS499-DataSci-course-materials/materials-sp24-colab/lec_demos/')

In [ ]:
# Import packages and other things needed
# Don't change this cell; Just run this cell
# If you restart colab, make sure to run this cell again after the first ones above^

from datascience import *
import numpy as np
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")
plt.rcParams["patch.force_edgecolor"] = True

## Learning Objectives


Topics:
- Reviewing table methods like `join`, `group`, `pivot` and `apply`
- conditional statements

## A Join Example ##

In [ ]:
# One table
full = Table.read_table('nc-est2019-agesex-res.csv')
census = full.select('SEX', 'AGE', 'POPESTIMATE2019')
census.show(3)

In [ ]:
# A second table
sex_codes = Table().with_columns(
    'SEX CODE', make_array(0, 1, 2),
    'CODE DEFINITION', make_array('All', 'Selected Male', 'Selected Female')
)
sex_codes

In [ ]:
# Join the tables
sex_codes.join('SEX CODE', census, 'SEX').sort('AGE').show(3)

## Bike Sharing ##

In [ ]:
# Read in the trip data from csv file
trip = Table.read_table('trip.csv')
trip.show(3)

## Distribution of Durations ##

In [ ]:
# Visualize the durations (seconds)
trip.hist('Duration')

Why does the histogram look like this?

In [ ]:
# Sort by duration with largest values first
trip.sort('Duration', descending=True)

In [ ]:
# Just trips under 30 mins (free ride length)
commute = trip.where('Duration', are.below(60*30)) # 60 secs * 30 mins
commute.hist('Duration')

Why are there not any values right after 0?<br>Hint: there was a minimum time limit

In [ ]:
# Now specify bins instead of letting hist automatically pick it
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

Based on the histogram, can you approximate the percent of people who have a ride duration between 250 and 500 seconds? <br>
Hint: Think about what formula you need and what information you have.

In [ ]:
#@title Example Solution
# Approx percent of people who have
# a ride duration between 250 and 500 seconds
# "between" = [250, 500)

(500-250) * 0.15

Calculating the exact value:

In [ ]:
# Get the number of rows using .where
commute.where('Duration', are.between(250, 500)).num_rows

In [ ]:
# take number of trips from the duration window and divide by the number of rows in the table
# gives us percentage in that interval
129079 / commute.num_rows * 100

In [ ]:
# Same code as above
commute.hist('Duration', bins=np.arange(0, 1800, 250), unit='Second')

In [ ]:
 # Now put in a number of bins for comparison
commute.hist('Duration', bins=60, unit='Second')

## Start and End Stations ##

In [ ]:
# Most common start station
# group finds unique values of the Start Station and gets the count of those
starts = commute.group('Start Station').sort('count', descending=True)
starts

What could we use to look at trips between stations?

In [ ]:
#@title Example Solution

# Using group
commute.group(['Start Station', 'End Station'])

In [ ]:
#@title Example Solution

# Using pivot
commute.pivot('Start Station', 'End Station')

Why are there so many zeros? Anything you remember about what portion of the dataset we are using?

Let's look at the average duration for the combination of the stations.

In [ ]:
# Average durations of trips between stations
commute.pivot('Start Station', 'End Station', values='Duration', collect=np.average)

In [ ]:
# Time it should take- direct without stopping
commute.pivot('Start Station', 'End Station', values='Duration', collect=min)

What if we wanted to put multiple numbers in our table that include the min and the max?

In [ ]:
# Define our own function and test it out
def min_and_max(values):
  return str(min(values)) + "," + str(max(values))

In [ ]:
# Time it should take- direct without stopping
commute.pivot('Start Station', 'End Station', values='Duration', collect=min_and_max)

How can we find the fastest trip ever between each pair of stations using group?

In [ ]:
#@title Example Solution

duration = trip.select('Start Station', 'End Station', 'Duration')
duration

In [ ]:
#@title Example Solution

shortest = duration.group(['Start Station', 'End Station'], min)
shortest.show(5)

## Discussion question

Find the 5 stations closest to Civic Center BART by minimum trip time.

In [ ]:
#@title Example Solution
from_cc = shortest.where('Start Station', are.containing('Civic Center BART')).sort('Duration min')

from_cc.take(np.arange(5)).column('End Station')

## Maps ##

In [ ]:
# Geographical data on the stations (dropping some columns we don't need)
stations = Table.read_table('station.csv').drop(4, 6)
stations

In [ ]:
sf_stations = stations.where('landmark', are.equal_to('San Francisco'))
sf_stations_map_data = (sf_stations
 .select('lat', 'long', 'name')
 .relabeled('name', 'labels'))
sf_stations_map_data.show(3)

In [ ]:
Marker.map_table(sf_stations_map_data)

In [ ]:
Circle.map_table(sf_stations_map_data)

## Conditionals and Iteration

## Comparison ##

We have already seen basic boolean expressions before

In [ ]:
3 > 1

In [ ]:
type(3 > 1)

In [ ]:
type(True)

In [ ]:
# What is the error here?
true

Recall that the single `=` is assignment.
 Thus the following error:

In [ ]:
3 = 3

Equality:

In [ ]:
3 == 3.0

Inequality (not equal to):

In [ ]:
10 != 2

Using variables in boolean expressions:

In [2]:
x = 14
y = 3

In [ ]:
x > 15

In [ ]:
12 < x

In [ ]:
x < 20

Compound boolean expressions:

In [ ]:
12 < x < 20

In [ ]:
(12 < x) and (x < 20)

In [ ]:
(12 < x) and (x < 5)

In [ ]:
(12 < x) or (x < 5)

In [ ]:
10 < x - y < 13

In [ ]:
x > 13 and y < 3.14159

(The comparison `12 < x < 20` is equivalent to `12 < x and x < 20`.)

## Comparisons with arrays

In [ ]:
# An array of pets
pets = make_array('cat', 'cat', 'dog', 'cat', 'dog', 'rabbit')

In [ ]:
# Here it checks whether each element in the array is equal (==) to cat
pets == 'cat'

How many cats?

In [ ]:
sum(pets == 'cat')

Math with booleans

In [ ]:
# Converts to true and false and then counts them as one or zero
np.mean(pets == 'cat')

In [ ]:
# An assignment statement with equality operation
is_cat = pets == 'cat'
is_cat

In [ ]:
is_cat * 2 - 1

In [ ]:
sum(make_array(True, True, False, True, False, False))

In [ ]:
np.count_nonzero(pets == 'dog')

In [ ]:
x = np.arange(20, 31)
x

In [ ]:
x > 28

In [ ]:
sum(x > 28)

### Extra: More ways to work with data and use the maps plotting features

Map all stations within 4 minutes (minimum ride time) of Civic Center.

In [ ]:
from_cc.show(3)

In [ ]:
sf_stations_map_data.show(3)

In [ ]:
#@title Example Solution
close_cc = from_cc.where('Duration min', are.below(4 * 60))

close_markers = (sf_stations_map_data
      .join('labels', close_cc, 'End Station')
      .select('lat', 'long', 'labels'))

Marker.map_table(close_markers)

Choose marker colors by the minimum time from Civic Center

In [ ]:
minutes = np.round(from_cc.column("Duration min") / 60)
print(min(minutes), max(minutes))

In [ ]:
colors = Table().with_columns(
    "minutes", np.arange(15),
    "colors",  ["darkblue", "blue", "lightblue",
                "darkgreen", "green", "lightgreen",
                "orange", "darkred", "red",
                "gray", "gray", "gray",
                "gray", "gray", "gray"])
colors_cc = (from_cc
 .with_column("Minutes", minutes)
 .join("Minutes", colors, "minutes"))

colored_markers = (sf_stations_map_data
      .join('labels', colors_cc, 'End Station')
      .select('lat', 'long', 'labels', 'colors'))
Marker.map_table(colored_markers)

In [ ]:
station_starts = stations.join('name', starts, 'Start Station')
station_starts.show(3)

In [ ]:
sf_starts = station_starts.relabeled(
    'name', 'labels').where('landmark', 'San Francisco')

In [ ]:
sf_starts_map_data = sf_starts.select('lat', 'long', 'labels').with_columns(
    'colors', 'blue',
    'areas', sf_starts.column('count') / 10
)
sf_starts_map_data.show(3)

In [ ]:
Circle.map_table(sf_starts_map_data)

### Saving
Remember to save your notebook before closing.
Choose **Save** (and make sure you've already saved a copy in your drive) from the **File** menu.